In [5]:
!pip install backtesting
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

--2024-04-25 13:19:06--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-04-25 13:19:07--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://cfhcable.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2024-04-25 13:19:07--  http://cfhcable.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1
Resolvi

In [8]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from backtesting import Backtest, Strategy
from sklearn.mixture import GaussianMixture
import talib as ta


# รับข้อมูลประวัติของ BTC-USD
data = yf.download('BTC-USD', start='2015-01-01', end='2024-04-25')

# สร้างคอลัมน์ 'Returns' และ 'Range'
data['Returns'] = data['Close'].pct_change()
data['Range'] = (data['High'] - data['Low']) / data['Low']
data.dropna(inplace=True)

# แบ่งข้อมูลเป็นชุดการฝึกอบรมและทดสอบ
train_data = data.iloc[:int(len(data) * 0.8)]
test_data = data.iloc[int(len(data) * 0.2):]

# สร้างโมเดล Gaussian Mixture Model (GMM) ด้วย n_components = 3
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(train_data[['Returns', 'Range']])

# ทำการคาดการณ์ hidden states จากชุดทดสอบ
test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])

# เพิ่มเงินสดเริ่มต้นเป็น 100000 USD
initial_cash = 100000  # ใช้เงินสดเริ่มต้นที่เพิ่มขึ้น

# สร้างกลยุทธ์รวมที่อ้างอิงถึงผลลัพธ์ของ GMM และ ADX
class CombinedStrategy(Strategy):
    def init(self):
        # คำนวณ ADX ด้วย TA-Lib
        self.adx = self.I(ta.ADX, self.data.High, self.data.Low, self.data.Close, timeperiod=14)

    def next(self):
        if  self.adx > 14:
            self.buy(size=0.1)

        elif self.data.hidden_state == 2 :
            self.sell(size=0.1)

# สร้างการ backtest ด้วยกลยุทธ์รวมและข้อมูลชุดทดสอบ
bt = Backtest(test_data, CombinedStrategy, cash=initial_cash, commission=0.001)

# รันการ backtest และพิมพ์สถิติ
stats = bt.run()
print(stats)

# แสดงผลการ backtest
bt.plot()
plt.show()


[*********************100%%**********************]  1 of 1 completed
<ipython-input-8-a8c157772160>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'hidden_state'] = gmm.predict(test_data[['Returns', 'Range']])


Start                     2016-11-12 00:00:00
End                       2024-04-24 00:00:00
Duration                   2720 days 00:00:00
Exposure Time [%]                   98.934215
Equity Final [$]               7487750.726682
Equity Peak [$]                8266181.527276
Return [%]                        7387.750727
Buy & Hold Return [%]             9016.592058
Return (Ann.) [%]                    78.41402
Volatility (Ann.) [%]              146.423209
Sharpe Ratio                          0.53553
Sortino Ratio                        1.618099
Calmar Ratio                         0.943533
Max. Drawdown [%]                  -83.106822
Avg. Drawdown [%]                  -11.179922
Max. Drawdown Duration     1080 days 00:00:00
Avg. Drawdown Duration       52 days 00:00:00
# Trades                                   29
Win Rate [%]                            100.0
Best Trade [%]                    8515.445726
Worst Trade [%]                   6252.199811
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(
